<a href="https://colab.research.google.com/github/Evgeniya371/PRA3024/blob/main/Task_2_(Energy_ratios).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='green'> Task 2 (Energy distribution)

###Run each cell below one by one

Run without changes

In [ ]:
# This cell imports all important packages
#!pip install streamlit_jupyter
import math
from scipy.special import jv, iv, jvp, ivp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import streamlit as st
import matplotlib
from mpl_toolkits import mplot3d
from matplotlib import cm
import scipy.integrate as integrate
from scipy.integrate import dblquad

Run without changes

In [5]:
# References:
# https://math.libretexts.org/Under_Construction/Numerical_Methods_with_Applications_(Kaw)/3%3A_Nonlinear_Equations/3.03%3A_Bisection_Methods_for_Solving_a_Nonlinear_Equation
# https://pythonnumericalmethods.berkeley.edu/notebooks/chapter19.03-Bisection-Method.html

def bisection_algorithm(f, x_l, x_u, tolerance, it_num, iter=None):

  # f - function
  # x_l, x_u are initial guesses (lower bound and upper bound respectively)
  # tolerance - percent relative error tolerance specified by user
  # it_num - maximum numer of iterations allowed (it is introduced to prevent infinite loops)
  # iter - iteration counter

    if iter is None:
        iter = 0


  # Check if the upper and lower bounds are roots themselves:
    if f(x_l) == 0:
        return x_l, iter
    if f(x_u) == 0:
        return x_u, iter


  # Check if root is bracketed:
    if f(x_l)*f(x_u)>0:
        #raise Exception("The function does not change sign. No root between x_l and x_u!")
        return None, iter


  # Find a mid-point (estimate of the root):
    x_m = (x_l + x_u)/2


  # Check the following:

    if f(x_l)*f(x_m)==0:
    # The root x_m is found. Terminate the algorithm and report the root.
       return x_m, iter


    if f(x_l)*f(x_m)<0:
    # The root lies between x_l and x_u, it follows that x_l = x_l and x_u = x_m
      x_l = x_l
      x_u = x_m


    if f(x_l)*f(x_m)>0:
    # The root lies between x_m and x_u, it follows that x_l = x_m and x_u = x_u
      x_l = x_m
      x_u = x_u


   # Update the mid-point:
    x_m_old=x_m
    x_m = (x_l + x_u)/2


   # Find the absolute realtive approximate error eps_abs
   # Formula for eps_abs  is |e_a|=|x_m_new-x_m_old)/x_m_new|*100%
    eps_abs=abs((x_m-x_m_old)/x_m)*100

    if eps_abs>tolerance:
     # Aplly the recursive method
      iter += 1
      return bisection_algorithm(f, x_l, x_u, tolerance, it_num, iter=iter)
    else:
     # Accept the estimate of the root otherwise
      return x_m, iter

      # You can print the string shown below to see the number of iterations needed to find each root
      #'Estimated root is %s. \n Number of iterations is %s.'%(x_m, iter)

Run without changes

In [6]:

def roots_bisection_method(f, a, b, tolerance, it_num, step):

  # f - function
  # a and b are the endpoints of the interval. Roots are searched within this interval.
  # tolerance - percent relative error tolerance specified by user
  # it_num - maximum numer of iterations allowed (it is introduced to prevent infinite loops)
  # step - stepsize which is used to search for the roots. It breaks interval [a, b] into small intevals

    roots=[] # This list stores all found roots

    x_l=a
    x_u=a+step
    while x_l<b: # Go along the x axis and break it into small intervals of size 'step' until you reach endpoint b
          root, iter=bisection_algorithm(f, x_l, x_u, tolerance, it_num)
          roots.append(root)
          x_l=x_u
          x_u=x_l+step
    roots=list(filter(lambda item: item is not None, roots)) # This line fileters 'None' values contained in the list roots

    return roots

Run without changes

In [7]:

# This function computes amplitude parameter  C_mn
def C_mn(lambda_mn, poisson_ratio, n):
    numerator=(lambda_mn**2)*jv(n, lambda_mn)+(1-poisson_ratio)*(lambda_mn*jvp(n, lambda_mn)-(n**2)*jv(n, lambda_mn))
    denominator=(lambda_mn**2)*iv(n, lambda_mn)-(1-poisson_ratio)*(lambda_mn*ivp(n, lambda_mn)-(n**2)*iv(n, lambda_mn))
    C_mn=numerator/denominator
    return C_mn

# This function computes frequency f_mn
def frequency_mn(lambda_mn, poisson_ratio, E, h, rho, a):
    D=E*(h**3)/(12*(1-poisson_ratio**2)) # D is flexural rigidity
    frequency_mn=(lambda_mn**2/(2*math.pi*a**2))*math.sqrt(D/(rho*h))
    return frequency_mn

# This function computes angular frequency omega_mn
def angular_frequency_mn(lambda_mn, poisson_ratio, E, h, rho, a):
    angular_frequency_mn=2*math.pi*frequency_mn(lambda_mn, poisson_ratio, E, h, rho, a)
    return angular_frequency_mn

# This function creates a list of eqully speced integers m
# The user should specify up to which number the values m should be computed
def m_list(m):
    return [item for item in range(0, m+1)]

# This function creates a list of eqully speced integers n
# The user should specify up to which number the values n should be computed
def n_list(n):
    return [item for item in range(0, n+1)]





*   Run the cell below

*   You may adjust the decimal places which are to be displayed in the tables. For this, set the desired number of decimal places in the following line:  
s = df.style.format('{:.2f}.')





In [8]:

# This function creates a table of modes
def table_lambda_m_n_values(n_num, m_num, poisson_ratio, a, b, tolerance, it_num, step):

  # n - number of n modes to be computed (corresponds to a number of columns). Each n is number of nodal diameters
  # m - number of m modes to be computed (corresponds to a number of rows). Each m is number of nodal circles
  # poisson_ratio - poisson ratio
  # the parameters listed below are related to the bisection method
  # a and b are the endpoints of the interval. Roots are searched within this interval.
  # tolerance - percent relative error tolerance specified by user
  # it_num - maximum numer of iterations allowed (it is introduced to prevent infinite loops)
  # step - stepsize which is used to search for the roots. It breaks interval [a, b] into small intevals

    # These two lines create the lists of m and n values
    m_values=m_list(m_num)
    n_values=n_list(n_num)

    # Create a dataframe for the table
    df = pd.DataFrame(index=pd.Index(m_values, name='m'), columns=n_values)

    # Create an array to store the value
    lambda_array = np.empty((m_num+1, n_num+1), dtype=float)

    # For loops find list of m valus for each n
    for n in n_values:
         # Function f is (4)-(5) (Amabili et al, p. 686)
         func = lambda x, n, poisson_ratio: ((x ** 2) * jv(n, x) + (1 - poisson_ratio) * (x* jvp(n, x) - (n ** 2) * jv(n, x)))/((x ** 2) * iv(n, x) - (1 - poisson_ratio) * (x * ivp(n, x) - (n ** 2) * iv(n, x)))-((x ** 3) * jvp(n, x) + (n ** 2) * (1 - poisson_ratio) * (x* jvp(n, x) - jv(n, x)))/((x ** 3) * ivp(n, x) - (n ** 2) * (1 - poisson_ratio) * (x * ivp(n, x) - iv(n, x)))
         f = lambda x: func(x, n, poisson_ratio)
         roots = roots_bisection_method(f, a, b, tolerance, it_num, step)
         if n>1:
             roots=roots[1:]
         if n==0 or n==1:
             roots=[np.nan]+roots
         for m in m_values:
             lambda_array[m,n] = roots[m]              # Compute lambda
             df.at[m, n] = roots[m]     # Compute lambda

    file_name = "output.xlsx"
    df.to_excel(file_name, sheet_name=f"λ values") # The reslut can be exported as Exel table

    # Specify the decimal places to be displayed
    s = df.style.format('{:.2f}')

    cell_hover = {
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
    }

    s.set_caption("λ values for n of:")\
    .set_table_styles([{
    'selector': 'caption',
    'props': 'caption-side: top; font-size:1.7em;'
    }], overwrite=False)

    df = s.set_table_styles([cell_hover])


    return s, lambda_array # First returened element is the table, second element is the array of lambda_mn values




*   You can adjust the value of the ampltidue parameter A in the following line:  
A=0.1*C

*   Run the cell






In [9]:
# This function computes elastic energy

def elastic_energy(m, n, poisson_ratio, E, lambda_array, a, z_lower, z_upper):

    # Comute the parameters values for the selected (m,n) mode
    lambda_mn = lambda_array[m,n]
    C = C_mn(lambda_mn, poisson_ratio, n)
    A=0.00000001*C

    # Factors (depend on material parameters)
    C_1=E/(1 + poisson_ratio)      # Constant 1
    C_2=1/(2*(1 - poisson_ratio))  # Constant 2
    k=lambda_mn/a     # multiplication factor (constant)

    # Terms as functions of parameters n, k, A, C, r, theta
    pd_1 = lambda n, k, A, C, r, theta: A * np.cos(n*theta) * ((k**3) * (C*ivp(n+1, k*r)-jvp(n+1, k*r)) + (k**2) * (n/r) * (C*ivp(n, k*r)+jvp(n,k*r)) - k * (n/(r**2)) * (C*iv(n, k*r)+jv(n, k*r)))
    pd_2 = lambda n, k, A, C, r, theta: A * np.cos(n*theta) * (1/r) * k * (C*ivp(n, k*r)+jvp(n, k*r))
    pd_3 = lambda n, k, A, C, r, theta: - A * ((n/r)**2) * np.cos(n*theta) * (C*iv(n, k*r)+jv(n, k*r))
    pd_4 = lambda n, k, A, C, r, theta: - A * n * np.sin(n*theta) * ((k/r) * (C*ivp(n, k*r)+jvp(n,k*r)) - 1/(r**2) * (C*iv(n, k*r) + jv(n, k*r)))

    # The infinitesimal total elastic energy (Amato et al, 2022)
    dE =  lambda z, r, theta: C_1*(C_2*(pd_1(n, k, A, C, r, theta)+pd_2(n, k, A, C, r, theta)+pd_3(n, k, A, C, r, theta))**2-(pd_1(n, k, A, C, r, theta)*(pd_2(n, k, A, C, r, theta)+pd_3(n, k, A, C, r, theta))-(pd_4(n, k, A, C, r, theta)**2)))*(z**2)*r

    # Solve triple integral to find E elastic
    # First integral is with respect to 'theta', second integral is with respect to radius 'r', third is with respect to 'z'
    E=integrate.tplquad(dE, 0, 2*np.pi, 0, a, z_lower, z_upper)  # Cylindrical coordiates (the origin is in the center of the lowest surface of the cylinder)

    return E[0]


Specify how many modes should be computed in the cell below. Also set the parameters values there.

In [ ]:
# Columns and rows
n_num = 1    # the values of the table are computed up to this number of n
m_num = 1   # the values of the table are computed up to this number of m

# The values of parameters are found in: (Granata et al, 2020)

# Substrate (fused silica):
Y_s= 71.2*10**9           # substate's Young's modulus (Pa)
poisson_ratio_s = 0.16    # substate's poisson ratio
density_s = 2220          # substate's mass density (kg/m^3)

# Coating (tantala):
Y_c = 121*10**9           # coating's Young's modulus (Pa)
poisson_ratio_c = 0.3     # coating's poisson ratio
density_c = 7400          # coating's mass density (kg/m^3)

# Geometry:
radius = 50/2*10**(-3)  # radius of the plate (m)
t_s = 1*10**(-3)        # thickness of the substrate (m)
t_c = 1*10**(-6)        # thickness of the coating (m)
t_tot = t_s+t_c         # thickness of the coated sample (m)

The next cell computes the frequency parameters, $\lambda_{mn}$,  resonance frequencies, $f_{mn}$, elastic energies of the substrate, $E_{s}$, and coating, $E_{c}$, and the corresponding dilution factor, $D$. The output is presented in tables.

* Run the cell
* The tables will be displayed below, but you can also download them from the folder: open the folder, and download the 'output.xslx' file. The first spreadsheet of the 'output.xslx' stores the $λ_{mn}$ values for each corresponding mode $(m,n)$. Other spreadsheets contain the computed parameters


In [ ]:
tab, lambda_array = table_lambda_m_n_values(n_num = n_num, m_num = m_num, poisson_ratio = poisson_ratio_s, a = 0.1, b = 50, tolerance = 0.00001, it_num = 100, step = 1)

m_values=m_list(m_num)
n_values=n_list(n_num)

# For loops find list of m valus for each n
for m in m_values:

        # Create a dataframe for the table
        df = pd.DataFrame(index=n_values, columns=['m','n', 'f', 'E_s', 'E_c', 'E_tot', 'D'])


        for n in n_values:

           # Comute the parameters values for the selected (m,n) mode

           if (m==0 and n==0) or (m==0 and n==1):
              df.at[n, 'm'] = m
              df.at[n, 'n'] = n
              df.at[n, 'f'] = np.nan
              df.at[n, 'E_s'] = np.nan
              df.at[n, 'E_c'] = np.nan
              df.at[n, 'E_tot'] = np.nan
              df.at[n, 'D'] = np.nan

           else:
              # Compute substrate and coating contributions
              E_s = elastic_energy(m = m, n = n, poisson_ratio = poisson_ratio_s, E = Y_s, lambda_array = lambda_array, a = radius, z_lower = 0, z_upper = t_s)
              E_c = elastic_energy(m = m, n = n, poisson_ratio = poisson_ratio_c, E = Y_c, lambda_array = lambda_array, a = radius, z_lower = t_s , z_upper = t_tot)
              # Compute total elastic energy
              E_tot = E_s + E_c
              # Frequency
              frequency = frequency_mn(lambda_mn = lambda_array[m,n], poisson_ratio=poisson_ratio_s, E = Y_s, rho = density_s, h = t_s, a = radius)
              # Dilution factor
              D = E_c/E_tot

              df.at[n, 'm'] = m
              df.at[n, 'n'] = n
              df.at[n, 'f'] = frequency
              df.at[n, 'E_s'] = E_s
              df.at[n, 'E_c'] = E_c
              df.at[n, 'E_tot'] = E_tot
              df.at[n, 'D'] = D


        # Formating of the tables:

        s=df.style.hide(axis='index')

        cell_hover = {
        'selector': 'td:hover',
        'props': [('background-color', '#ffffb3')]
        }

        s.set_caption(f"m = {m}")\
        .set_table_styles([{
        'selector': 'caption',
        'props': 'caption-side: top; font-size:1.7em;'
        }], overwrite=False)

        s.set_table_styles([cell_hover])

        # The computed tables are streamed into the "output.xlsx" file
        file_name = "output.xlsx"
        with pd.ExcelWriter(file_name, mode='a') as writer:
              df.to_excel(writer, sheet_name=f"m = {m}")

        # Display the tables
        display(s)